In [1]:
%matplotlib widget

In [2]:
import sys
import os
import time
import math as math
from datetime import datetime
import numpy as np
import astropy as ast
from astropy.io import fits
import astropy.io.fits as pyfits
from astropy.table import Table, hstack, vstack, Column
from astropy.stats import sigma_clipped_stats
from astropy.coordinates import Angle
from astropy.time import Time
from photutils.aperture import EllipticalAnnulus, aperture_photometry
import matplotlib
import matplotlib.pyplot as plt
import scipy as sp
import scipy.integrate
import scipy.constants
import scipy.special
from scipy import integrate
from scipy.optimize import curve_fit
import scipy.interpolate
from scipy.interpolate import CubicSpline
import glob2
import shutil
from IPython.display import clear_output
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.mplot3d import axes3d
from tqdm.notebook import tqdm as tqdm

import Utility3_4 as ut

pi = sp.constants.pi
k= sp.constants.k
q_e = sp.constants.e

DimY = 1096
DimX = 1936

DisectString = '\1/'
BackSlash = DisectString[0]
ForwardSlash = '/'

In [17]:
#Making a list of all set names
SetNamesList = ['X0_Y0','X0_YM1','X0_YM2','X0_YM3','X0_YP1','X0_YP2','X0_YP3','XM2_Y0','XM2_YM1','XM2_YP1','XP2_Y0','XP2_YM1','XP2_YP1','XP4_Y0']
#incomplete because I have to do the steady stating for the gain observations then add them too.

In [18]:
#Analyzing the most recent sets of Dark Frames
DimY = 1096
DimX = 1936
hotsig = 5.0 #Number of stdevs above a frame's median a pixel must be to be considered hot.
hotfrac = 0.1 #The fraction of the frames in a given set a certain pixel must be hot in order to be recorded as hot in the HotsArray

DarkFrameName = 'PrelimMeanCS_Dark'
HotsArrayName = 'PrelimHotsArray'

HotCount = []
HotFrac = []
FrameMediansList = []
FrameStdsList = []

for i in range(len(SetNamesList)):
    SetName = SetNamesList[i]
    
    #OpenDirectory
    DataDirectory = 'SteadyStateDataSets/' + SetName + '\CS_Darks/'
    filepatternOpen = '*.FTS'
    
    #SaveDirectory
    SaveDirectory = 'SteadyStateDataSets/' + SetName + ForwardSlash 
    
    #print(datadirectoryOpen + folderOpen)
    #print(datadirectorySave + folderSave)
    #print(SaveDirectory)
    MeanDarkFrame, HotsArray, FrameMedians, FrameStds = ut.ReturnMasterDarkAndHotsArrayExtra(DataDirectory,filepatternOpen,DimX,DimY,hotsig,hotfrac)
    
    #np.savetxt(SaveDirectory + DarkFrameName +'.csv', MeanDarkFrame, delimiter=',')
    #np.savetxt(SaveDirectory + HotsArrayName +'.csv', HotsArray, delimiter=',')
    
    #print(SetName + ' Preliminary Hot Pixel Count' + ": " + str(np.sum(HotsArray)))
    
    HotCount.append(np.sum(HotsArray))
    HotFrac.append(np.sum(HotsArray)/(DimX*DimY))
    FrameMediansList.append(FrameMedians)
    FrameStdsList.append(FrameStds)
    
#Analyzing old sets of Dark Frames
ComboSetNamesList = SetNamesList

#for entry in SetNamesList:
    #ComboSetNamesList.append[entry]

DataDirectory = 'OldCS300msDark/'
filepatternOpen = '*.FTS'

MeanDarkFrame, HotsArray, FrameMedians, FrameStds = ut.ReturnMasterDarkAndHotsArrayExtra(DataDirectory,filepatternOpen,DimX,DimY,hotsig,hotfrac)

HotCount.append(np.sum(HotsArray))
HotFrac.append(np.sum(HotsArray)/(DimX*DimY))
FrameMediansList.append(FrameMedians)
FrameStdsList.append(FrameStds)

ComboSetNamesList.append('OldCS300msDark')

DataDirectory = 'OldCS1000msDark/'
filepatternOpen = '*.FTS'

MeanDarkFrame, HotsArray, FrameMedians, FrameStds = ut.ReturnMasterDarkAndHotsArrayExtra(DataDirectory,filepatternOpen,DimX,DimY,hotsig,hotfrac)

HotCount.append(np.sum(HotsArray))
HotFrac.append(np.sum(HotsArray)/(DimX*DimY))
FrameMediansList.append(FrameMedians)
FrameStdsList.append(FrameStds)

ComboSetNamesList.append('OldCS1000msDark')


  0%|          | 0/246 [00:00<?, ?it/s]

  0%|          | 0/36 [00:00<?, ?it/s]

  0%|          | 0/48 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/49 [00:00<?, ?it/s]

...

  0%|          | 0/49 [00:00<?, ?it/s]

  0%|          | 0/49 [00:00<?, ?it/s]

  0%|          | 0/48 [00:00<?, ?it/s]

  0%|          | 0/99 [00:00<?, ?it/s]

  0%|          | 0/99 [00:00<?, ?it/s]

In [25]:
for i in range(len(ComboSetNamesList)):
    SetName = ComboSetNamesList[i]
    MeanFrameMedian = np.mean(FrameMediansList[i])
    MeanFrameStd = np.mean(FrameStdsList[i])
    print("_________________________")
    print("")
    print(" Set: "+ SetName)
    print("-------------------------")
    print(" Hot Pixel Count: " + str(HotCount[i]))
    print(" Hot Pixel Fraction: " + str(round(HotFrac[i]*100,4))+"%")
    print(" Mean Frame Median: " + str(round(MeanFrameMedian,2)))
    print(" Mean Frame Std: " + str(round(MeanFrameStd,2)))
    
print("_________________________")

_________________________

 Set: X0_Y0
-------------------------
 Hot Pixel Count: 270.0
 Hot Pixel Fraction: 0.0127%
 Mean Frame Median: 4.0
 Mean Frame Std: 3.1
_________________________

 Set: X0_YM1
-------------------------
 Hot Pixel Count: 270.0
 Hot Pixel Fraction: 0.0127%
 Mean Frame Median: 4.0
 Mean Frame Std: 3.07
_________________________

 Set: X0_YM2
-------------------------
 Hot Pixel Count: 272.0
 Hot Pixel Fraction: 0.0128%
 Mean Frame Median: 4.0
 Mean Frame Std: 3.12
_________________________

 Set: X0_YM3
-------------------------
 Hot Pixel Count: 270.0
 Hot Pixel Fraction: 0.0127%
 Mean Frame Median: 4.0
 Mean Frame Std: 3.11
_________________________

 Set: X0_YP1
-------------------------
 Hot Pixel Count: 270.0
 Hot Pixel Fraction: 0.0127%
 Mean Frame Median: 4.0
 Mean Frame Std: 3.1
_________________________

 Set: X0_YP2
-------------------------
 Hot Pixel Count: 272.0
 Hot Pixel Fraction: 0.0128%
 Mean Frame Median: 4.0
 Mean Frame Std: 3.08
_____________

Seeing if the new hots arrays have hotpixels that the old arrays didn't have.

In [27]:
DataDirectory = 'OldCS300msDark/'
filepatternOpen = '*.FTS'
SaveDirectory = 'OldHotsArrays/'

MeanDarkFrame, HotsArray, FrameMedians, FrameStds = ut.ReturnMasterDarkAndHotsArrayExtra(DataDirectory,filepatternOpen,DimX,DimY,hotsig,hotfrac)

np.savetxt(SaveDirectory + 'Old300msHotsArray' +'.csv', HotsArray, delimiter=',')

DataDirectory = 'OldCS1000msDark/'
filepatternOpen = '*.FTS'

MeanDarkFrame, HotsArray, FrameMedians, FrameStds = ut.ReturnMasterDarkAndHotsArrayExtra(DataDirectory,filepatternOpen,DimX,DimY,hotsig,hotfrac)

np.savetxt(SaveDirectory + 'Old1000msHotsArray' +'.csv', HotsArray, delimiter=',')


  0%|          | 0/99 [00:00<?, ?it/s]

  0%|          | 0/99 [00:00<?, ?it/s]

In [3]:
#Make the prelim HotsArray for X0_YP2_981eV
SetName = 'X0_YP2_981eV'
DataDirectory = 'SteadyStateDataSets/Gain_Data/' + SetName + '\CS_Darks/'
filepatternOpen = '*.FTS'

DimY = 1096
DimX = 1936
hotsig = 5.0 #Number of stdevs above a frame's median a pixel must be to be considered hot.
hotfrac = 0.1 #The fraction of the frames in a given set a certain pixel must be hot in order to be recorded as hot in the HotsArray

DarkFrameName = 'PrelimMeanCS_Dark2'
HotsArrayName = 'PrelimHotsArray'

#SaveDirectory
SaveDirectory = 'SteadyStateDataSets/Gain_Data/' + SetName + ForwardSlash 

MeanDarkFrame, HotsArray, FrameMedians, FrameStds = ut.ReturnMasterDarkAndHotsArrayExtra(DataDirectory,filepatternOpen,DimX,DimY,hotsig,hotfrac)

np.savetxt(SaveDirectory + DarkFrameName +'.csv', MeanDarkFrame, delimiter=',')
np.savetxt(SaveDirectory + HotsArrayName +'.csv', HotsArray, delimiter=',')

  0%|          | 0/44 [00:00<?, ?it/s]

C:\Users\Colin Packard\PythonWorkspace\Argonne12_01\Utility3_4.py:290: RuntimeWarning: invalid value encountered in sqrt
  StdDarkFrame = np.sqrt((SumDarkSquared/len(ImagesList)) - MeanDarkFrame**2)


In [7]:
SetNamesList = ['X0_Y0','X0_YM1','X0_YM2','X0_YM3','X0_YP1','X0_YP2','X0_YP3','XM2_Y0','XM2_YM1','XM2_YP1','XP2_Y0','XP2_YM1','XP2_YP1','XP4_Y0']
#Load the old hots arrays
LoadDirectory = 'OldHotsArrays/'
Old300msHotsArray = np.loadtxt(LoadDirectory + 'Old300msHotsArray' +'.csv', delimiter=',')
Old1000msHotsArray = np.loadtxt(LoadDirectory + 'Old1000msHotsArray' +'.csv', delimiter=',')
ComboOldHotsArray = np.where(Old1000msHotsArray == 1,1,Old300msHotsArray)

NewComboHotsArray = np.zeros(np.shape(Old300msHotsArray)) 

for i in range(len(SetNamesList)):
    
    SetName = SetNamesList[i]
    HotsArrayName = 'PrelimHotsArray'
    LoadDirectory = 'SteadyStateDataSets/' + SetName + ForwardSlash 
    
    HotsArray = np.loadtxt(LoadDirectory + HotsArrayName +'.csv', delimiter=',')
    
    #Combining all of the unique hotpixels found in the new dataset.
    NewComboHotsArray = np.where(HotsArray == 1,1,NewComboHotsArray)
    
    CheckArray300 = np.zeros(np.shape(Old300msHotsArray))
    CheckArray300 = np.where(Old300msHotsArray == 1,0,HotsArray)
    CheckArray1000 = np.zeros(np.shape(Old300msHotsArray))
    CheckArray1000 = np.where(Old1000msHotsArray == 1,0,HotsArray)
    CheckArrayCombo = np.zeros(np.shape(Old300msHotsArray))
    CheckArrayCombo = np.where(ComboOldHotsArray == 1,0,HotsArray)
    
    NewUniqueHotPixels300 = np.sum(CheckArray300)
    NewUniqueHotPixels1000 = np.sum(CheckArray1000)
    NewUniqueHotPixels = np.sum(CheckArrayCombo)
    
    print("_________________________")
    print("")
    print(" Set: "+ SetName)
    print("-------------------------")
    print(" Number of New Hot Pixels: " + str(NewUniqueHotPixels))
    print(" Number of New Hot Pixels 300ms: " + str(NewUniqueHotPixels300))
    print(" Number of New Hot Pixels 1000ms: " + str(NewUniqueHotPixels1000))
    


SetName = 'X0_YP2_981eV'
HotsArrayName = 'PrelimHotsArray'
LoadDirectory = 'SteadyStateDataSets/Gain_Data/' + SetName + ForwardSlash 
    
HotsArray = np.loadtxt(LoadDirectory + HotsArrayName +'.csv', delimiter=',')

NewComboHotsArray = np.where(HotsArray == 1,1,NewComboHotsArray)

CheckArray300 = np.zeros(np.shape(Old300msHotsArray))
CheckArray300 = np.where(Old300msHotsArray == 1,0,HotsArray)
CheckArray1000 = np.zeros(np.shape(Old300msHotsArray))
CheckArray1000 = np.where(Old1000msHotsArray == 1,0,HotsArray)
CheckArrayCombo = np.zeros(np.shape(Old300msHotsArray))
CheckArrayCombo = np.where(ComboOldHotsArray == 1,0,HotsArray)

NewUniqueHotPixels300 = np.sum(CheckArray300)
NewUniqueHotPixels1000 = np.sum(CheckArray1000)
NewUniqueHotPixels = np.sum(CheckArrayCombo)

print("_________________________")
print("")
print(" Set: "+ SetName)
print("-------------------------")
print(" Number of New Hot Pixels: " + str(NewUniqueHotPixels))
print(" Number of New Hot Pixels 300ms: " + str(NewUniqueHotPixels300))
print(" Number of New Hot Pixels 1000ms: " + str(NewUniqueHotPixels1000))

#-------------------

CheckArray300 = np.zeros(np.shape(Old300msHotsArray))
CheckArray300 = np.where(Old300msHotsArray == 1,0,HotsArray)
CheckArray1000 = np.zeros(np.shape(Old300msHotsArray))
CheckArray1000 = np.where(Old1000msHotsArray == 1,0,HotsArray)
CheckArrayCombo = np.zeros(np.shape(Old300msHotsArray))
CheckArrayCombo = np.where(ComboOldHotsArray == 1,0,HotsArray)

NewUniqueHotPixels300 = np.sum(CheckArray300)
NewUniqueHotPixels1000 = np.sum(CheckArray1000)
NewUniqueHotPixels = np.sum(CheckArrayCombo)

print("_________________________")
print("")
print(" Set: "+ "Combined New HotsArray")
print("-------------------------")
print(" Number of New Hot Pixels: " + str(NewUniqueHotPixels))
print(" Number of New Hot Pixels 300ms: " + str(NewUniqueHotPixels300))
print(" Number of New Hot Pixels 1000ms: " + str(NewUniqueHotPixels1000))
print("_________________________")

SaveDirectory = 'SteadyStateDataSets/'
np.savetxt(SaveDirectory + 'ComboHotsArray' +'.csv', NewComboHotsArray, delimiter=',')

_________________________

 Set: X0_Y0
-------------------------
 Number of New Hot Pixels: 35.0
 Number of New Hot Pixels 300ms: 48.0
 Number of New Hot Pixels 1000ms: 35.0
_________________________

 Set: X0_YM1
-------------------------
 Number of New Hot Pixels: 35.0
 Number of New Hot Pixels 300ms: 48.0
 Number of New Hot Pixels 1000ms: 35.0
_________________________

 Set: X0_YM2
-------------------------
 Number of New Hot Pixels: 35.0
 Number of New Hot Pixels 300ms: 50.0
 Number of New Hot Pixels 1000ms: 35.0
_________________________

 Set: X0_YM3
-------------------------
 Number of New Hot Pixels: 35.0
 Number of New Hot Pixels 300ms: 48.0
 Number of New Hot Pixels 1000ms: 35.0
_________________________

 Set: X0_YP1
-------------------------
 Number of New Hot Pixels: 35.0
 Number of New Hot Pixels 300ms: 48.0
 Number of New Hot Pixels 1000ms: 35.0
_________________________

 Set: X0_YP2
-------------------------
 Number of New Hot Pixels: 35.0
 Number of New Hot Pixels 30

This seems to indicate that the hotpixels found from the old 1000 ms darks have all but 35 of the new hotpixels and that adding the hotpixels from the set of 300ms data frames doesn't change anything.

The new dark frames universially have 35 hotpixels not found in the old 1000 ms darks, and adding the hotpixels from the old 300 ms darks doesn't change anything. 

Combining the ComboNewHotsoArray with the Old1000msHotsArray

In [8]:
SetNamesList = ['X0_Y0','X0_YM1','X0_YM2','X0_YM3','X0_YP1','X0_YP2','X0_YP3','XM2_Y0','XM2_YM1','XM2_YP1','XP2_Y0','XP2_YM1','XP2_YP1','XP4_Y0']
#Load the old hots arrays
LoadDirectory = 'OldHotsArrays/'
Old300msHotsArray = np.loadtxt(LoadDirectory + 'Old300msHotsArray' +'.csv', delimiter=',')
Old1000msHotsArray = np.loadtxt(LoadDirectory + 'Old1000msHotsArray' +'.csv', delimiter=',')
ComboOldHotsArray = np.where(Old1000msHotsArray == 1,1,Old300msHotsArray)
LoadDirectory = 'SteadyStateDataSets/'
ComboNewHotsArray = np.loadtxt(LoadDirectory + 'ComboHotsArray' +'.csv', delimiter=',')

MasterHotsArray = np.zeros(np.shape(Old300msHotsArray))
MasterHotsArray = np.where(Old1000msHotsArray == 1,1,ComboNewHotsArray)

CheckArray300 = np.zeros(np.shape(Old300msHotsArray))
CheckArray300 = np.where(Old300msHotsArray == 1,0,MasterHotsArray)
CheckArray1000 = np.zeros(np.shape(Old300msHotsArray))
CheckArray1000 = np.where(Old1000msHotsArray == 1,0,MasterHotsArray)
CheckArrayCombo = np.zeros(np.shape(Old300msHotsArray))
CheckArrayCombo = np.where(ComboOldHotsArray == 1,0,MasterHotsArray)
NewUniqueHotPixels300 = np.sum(CheckArray300)
NewUniqueHotPixels1000 = np.sum(CheckArray1000)
NewUniqueHotPixels = np.sum(CheckArrayCombo)

print("_________________________")
print("")
print(" Set: "+ "MasterHotsArray")
print("-------------------------")
print(" Number of New Hot Pixels: " + str(NewUniqueHotPixels))
print(" Number of New Hot Pixels 300ms: " + str(NewUniqueHotPixels300))
print(" Number of New Hot Pixels 1000ms: " + str(NewUniqueHotPixels1000))
print("_________________________")

SaveDirectory = 'SteadyStateDataSets/'
np.savetxt(SaveDirectory + 'MasterHotsArray' +'.csv', MasterHotsArray, delimiter=',')

_________________________

 Set: MasterHotsArray
-------------------------
 Number of New Hot Pixels: 35.0
 Number of New Hot Pixels 300ms: 193.0
 Number of New Hot Pixels 1000ms: 35.0
_________________________


Trying to see what the hots arrays might look like if I used the HotThresh from the old datasets.

In [9]:
HotCount = np.sum(MasterHotsArray)
HotFrac = HotCount/(DimX*DimY)

print("_________________________")
print("")
print(" Set: "+ "MasterHotsArray")
print("-------------------------")
print(" Hot Pixel Count: " + str(HotCount))
print(" Hot Pixel Fraction: " + str(round(HotFrac*100,4))+"%")
print(" Number of New Hot Pixels: " + str(NewUniqueHotPixels))
print(" Number of New Hot Pixels 300ms: " + str(NewUniqueHotPixels300))
print(" Number of New Hot Pixels 1000ms: " + str(NewUniqueHotPixels1000))
print("_________________________")



_________________________

 Set: MasterHotsArray
-------------------------
 Hot Pixel Count: 760.0
 Hot Pixel Fraction: 0.0358%
 Number of New Hot Pixels: 35.0
 Number of New Hot Pixels 300ms: 193.0
 Number of New Hot Pixels 1000ms: 35.0
_________________________


### Making Master Dark Frames Now That I Have a Master Hots Array

In [15]:
def ReturnMeanDarkAndStats(datadirectory,filepattern,DimX,DimY,HotsArray):
    #DimY,DimX the dimensions of the images eg for 1080x1920 DimY=1080, DimX=1920
    
    #This Currently doesn't make a median dark frame because I havent figured out a way of doing that 
    #without making this code take roughly one-million times longer to run.
    
    #FileNameList = MakeFileNamesList(datadirectory,filepattern)
    FileNameList = glob2.glob(datadirectory + filepattern)
    ImagesList = ut.MakeImagesList(FileNameList,1)

    NumberOfFrames = len(ImagesList)
    SumDark = np.zeros((DimY,DimX))
    SumDarkSquared = np.zeros((DimY,DimX))
    SumExposureTime = 0
    
    ZerosArray = np.zeros((DimY,DimX))
    
    FrameMeans = []
    FrameMedians = []
    FrameStds = []
    
    #making master dark frame by finding the mean
    for i in tqdm(range(len(ImagesList))):
    #for all images in list of images
        
        #For Mean and Std Dark Frames
        image_and_metadata = ImagesList[i]
        image_data = image_and_metadata[0]
        SumDark = SumDark + image_data
        SumDarkSquared = SumDarkSquared + image_data**2
        
        SumExposureTime = SumExposureTime + float(image_and_metadata[1])
        
        FlatImageNoHots = []
        for x in range(DimX):
            for y in range(DimY):
                if HotsArray[y,x] != 1:
                    FlatImageNoHots.append(image_data[y,x])
        
        FlatImageNoHots = np.array(FlatImageNoHots)
        
        FrameMean = np.mean(FlatImageNoHots)
        FrameMedian = np.median(FlatImageNoHots)
        FrameStd = np.std(FlatImageNoHots)
        
        FrameMeans.append(FrameMean)
        FrameMedians.append(FrameMedian)
        FrameStds.append(FrameStd)
        
    FrameMeans = np.array(FrameMeans)
    FrameMedians = np.array(FrameMedians)
    FrameStds = np.array(FrameStds)
        
    MeanDarkFrame = SumDark/len(ImagesList)
    StdDarkFrame = np.sqrt((SumDarkSquared/len(ImagesList)) - MeanDarkFrame**2)
    
    MeanExposureTime = SumExposureTime/len(ImagesList)
    
    return MeanDarkFrame, FrameMeans, FrameMedians, FrameStds

In [16]:
SetNamesList = ['X0_Y0','X0_YM1','X0_YM2','X0_YM3','X0_YP1','X0_YP2','X0_YP3','XM2_Y0','XM2_YM1','XM2_YP1','XP2_Y0','XP2_YM1','XP2_YP1','XP4_Y0']
DarkFrameName = 'MeanCS_Dark'
HotsArrayName = 'MasterHotsArray'
DimY = 1096
DimX = 1936

HotsDirectory = 'SteadyStateDataSets/'
MasterHotsArray = np.loadtxt(HotsDirectory + HotsArrayName + '.csv', delimiter=',')

ComboSetNamesList = []

FrameMeansList = []
FrameMediansList = []
FrameStdsList = []

for i in range(len(SetNamesList)):
    SetName = SetNamesList[i]
    ComboSetNamesList.append(SetName)
    
    #OpenDirectory
    DataDirectory = 'SteadyStateDataSets/' + SetName + '\CS_Darks/'
    filepatternOpen = '*.FTS'
    
    #SaveDirectory
    SaveDirectory = 'SteadyStateDataSets/' + SetName + ForwardSlash 

    MeanDarkFrame, FrameMeans, FrameMedians, FrameStds = ReturnMeanDarkAndStats(DataDirectory,filepatternOpen,DimX,DimY,MasterHotsArray)
    
    FrameMeansList.append(FrameMeans)
    FrameMediansList.append(FrameMedians)
    FrameStdsList.append(FrameStds)
    
    np.savetxt(SaveDirectory + DarkFrameName +'.csv', MeanDarkFrame, delimiter=',')
    
    
#OpenDirectory
SetName = 'X0_YP2_981eV'
ComboSetNamesList.append(SetName)
DataDirectory = 'SteadyStateDataSets/Gain_Data/X0_YP2_981eV' + '\CS_Darks/'
filepatternOpen = '*.FTS'
#SaveDirectory
SaveDirectory = 'SteadyStateDataSets/Gain_Data/X0_YP2_981eV/'

MeanDarkFrame, FrameMeans, FrameMedians, FrameStds = ReturnMeanDarkAndStats(DataDirectory,filepatternOpen,DimX,DimY,MasterHotsArray)

FrameMeansList.append(FrameMeans)
FrameMediansList.append(FrameMedians)
FrameStdsList.append(FrameStds)

np.savetxt(SaveDirectory + DarkFrameName +'.csv', MeanDarkFrame, delimiter=',')

for i in range(len(ComboSetNamesList)):
    SetName = ComboSetNamesList[i]
    MeanFrameMean = np.mean(FrameMeansList[i])
    MeanFrameMedian = np.mean(FrameMediansList[i])
    MeanFrameStd = np.mean(FrameStdsList[i])
    print("_________________________")
    print("")
    print(" Set: "+ SetName)
    print("-------------------------")
    print(" Mean Frame Mean: " + str(round(MeanFrameMean,2)))
    print(" Mean Frame Median: " + str(round(MeanFrameMedian,2)))
    print(" Mean Frame Std: " + str(round(MeanFrameStd,2)))

  0%|          | 0/246 [00:00<?, ?it/s]

C:\Users\Colin Packard\AppData\Local\Temp\ipykernel_7808\3797429830.py:55: RuntimeWarning: invalid value encountered in sqrt
  StdDarkFrame = np.sqrt((SumDarkSquared/len(ImagesList)) - MeanDarkFrame**2)


  0%|          | 0/36 [00:00<?, ?it/s]

  0%|          | 0/48 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

...

  0%|          | 0/49 [00:00<?, ?it/s]

  0%|          | 0/49 [00:00<?, ?it/s]

  0%|          | 0/48 [00:00<?, ?it/s]

  0%|          | 0/44 [00:00<?, ?it/s]

_________________________

 Set: X0_Y0
-------------------------
 Mean Frame Mean: 3.55
 Mean Frame Median: 4.0
 Mean Frame Std: 1.24
_________________________

 Set: X0_YM1
-------------------------
 Mean Frame Mean: 3.55
 Mean Frame Median: 4.0
 Mean Frame Std: 1.24
_________________________

 Set: X0_YM2
-------------------------
 Mean Frame Mean: 3.52
 Mean Frame Median: 4.0
 Mean Frame Std: 1.25
_________________________

 Set: X0_YM3
-------------------------
 Mean Frame Mean: 3.54
 Mean Frame Median: 4.0
 Mean Frame Std: 1.24
_________________________

 Set: X0_YP1
-------------------------
 Mean Frame Mean: 3.55
 Mean Frame Median: 4.0
 Mean Frame Std: 1.24
_________________________

 Set: X0_YP2
-------------------------
 Mean Frame Mean: 3.52
 Mean Frame Median: 4.0
 Mean Frame Std: 1.25
_________________________

 Set: X0_YP3
-------------------------
 Mean Frame Mean: 3.54
 Mean Frame Median: 4.0
 Mean Frame Std: 1.24
_________________________

 Set: XM2_Y0
----------------

In [52]:
hotsig = 5.0
#For New X0_Y0 1000ms Darks
i=0
SetName = ComboSetNamesList[i]
MeanFrameMedian = np.mean(FrameMediansList[i])
MeanFrameStd = np.mean(FrameStdsList[i])

X0_Y0_HotThresh = MeanFrameMedian + MeanFrameStd*hotsig

print("_________________________")
print("")
print(" Set: "+ SetName + " (1000 ms CS Darks)")
print("-------------------------")
print(" Hot Pixel Count: " + str(HotCount[i]))
print(" Hot Pixel Fraction: " + str(round(HotFrac[i]*100,4))+"%")
print(" Mean Frame Median: " + str(round(MeanFrameMedian,2)))
print(" Mean Frame Std: " + str(round(MeanFrameStd,2)))
print(" HotThresh: " + str(X0_Y0_HotThresh))

#For Old 1000ms Darks
i=len(ComboSetNamesList)-1
SetName = ComboSetNamesList[i]
MeanFrameMedian = np.mean(FrameMediansList[i])
MeanFrameStd = np.mean(FrameStdsList[i])

OldHotThresh1000 = MeanFrameMedian + MeanFrameStd*hotsig

print("_________________________")
print("")
print(" Set: "+ SetName)
print("-------------------------")
print(" Hot Pixel Count: " + str(HotCount[i]))
print(" Hot Pixel Fraction: " + str(round(HotFrac[i]*100,4))+"%")
print(" Mean Frame Median: " + str(round(MeanFrameMedian,2)))
print(" Mean Frame Std: " + str(round(MeanFrameStd,2)))
print(" HotThresh: " + str(OldHotThresh1000))

#For Old 300ms Darks
i=len(ComboSetNamesList)-2
SetName = ComboSetNamesList[i]
MeanFrameMedian = np.mean(FrameMediansList[i])
MeanFrameStd = np.mean(FrameStdsList[i])

OldHotThresh300 = MeanFrameMedian + MeanFrameStd*hotsig

print("_________________________")
print("")
print(" Set: "+ SetName)
print("-------------------------")
print(" Hot Pixel Count: " + str(HotCount[i]))
print(" Hot Pixel Fraction: " + str(round(HotFrac[i]*100,4))+"%")
print(" Mean Frame Median: " + str(round(MeanFrameMedian,2)))
print(" Mean Frame Std: " + str(round(MeanFrameStd,2)))
print(" HotThresh: " + str(OldHotThresh300))


_________________________

 Set: X0_Y0 (1000 ms CS Darks)
-------------------------
 Hot Pixel Count: 270.0
 Hot Pixel Fraction: 0.0127%
 Mean Frame Median: 4.0
 Mean Frame Std: 3.1
 HotThresh: 19.51359865976557
_________________________

 Set: OldCS1000msDark
-------------------------
 Hot Pixel Count: 725.0
 Hot Pixel Fraction: 0.0342%
 Mean Frame Median: 3.01
 Mean Frame Std: 1.46
 HotThresh: 10.304468969727946
_________________________

 Set: OldCS300msDark
-------------------------
 Hot Pixel Count: 1759.0
 Hot Pixel Fraction: 0.0829%
 Mean Frame Median: 4.0
 Mean Frame Std: 1.14
 HotThresh: 9.712012184264198


In [46]:
def ReturnMasterDarkAndHotsArrayThresh(datadirectory,filepattern,DimX,DimY,HotThresh,hotfrac):
    #DimY,DimX the dimensions of the images eg for 1080x1920 DimY=1080, DimX=1920
    
    #hotsig: Number of stdevs above a frame's median a pixel must be to be considered hot.
    #hotfrac: The fraction of the frames in a given set a certain pixel must be hot in order to be recorded as hot in the HotsArray
    
    #This Currently doesn't make a median dark frame because I havent figured out a way of doing that 
    #without making this code take roughly one-million times longer to run.
    
    #FileNameList = MakeFileNamesList(datadirectory,filepattern)
    FileNameList = glob2.glob(datadirectory + filepattern)
    ImagesList = ut.MakeImagesList(FileNameList,1)

    NumberOfFrames = len(ImagesList)
    SumDark = np.zeros((DimY,DimX))
    SumDarkSquared = np.zeros((DimY,DimX))
    SumExposureTime = 0
    
    HotCounterArray = np.zeros((DimY,DimX))
    ZerosArray = np.zeros((DimY,DimX))
    #SumDark = np.zeros((1080,1920))
    
    
    #making master dark frame by finding the mean
    for i in tqdm(range(len(ImagesList))):
    #for all images in list of images
        
        #For Mean and Std Dark Frames
        image_and_metadata = ImagesList[i]
        image_data = image_and_metadata[0]
        SumDark = SumDark + image_data
        SumDarkSquared = SumDarkSquared + image_data**2
        
        SumExposureTime = SumExposureTime + float(image_and_metadata[1])
        
        #For finding Hot Pixels
        FrameMedian = np.median(image_data)
        FrameStd = np.std(image_data)
        
        #hotsig is the number of stdevs above median a pixel must be to be considered hot.
        #HotThresh = FrameMedian + hotsig*FrameStd
        HotsToCount = np.where(image_data > HotThresh,1,ZerosArray)
        
        HotCounterArray = HotCounterArray + HotsToCount


    MeanDarkFrame = SumDark/len(ImagesList)
    StdDarkFrame = np.sqrt((SumDarkSquared/len(ImagesList)) - MeanDarkFrame**2)
    HotsArray = np.where((HotCounterArray/len(ImagesList)) > hotfrac,1,ZerosArray)
    
    MeanExposureTime = SumExposureTime/len(ImagesList)
    
    return MeanDarkFrame, HotsArray

def ReturnMasterDarkAndHotsArrayExtraThresh(datadirectory,filepattern,DimX,DimY,HotThresh,hotfrac):
    #DimY,DimX the dimensions of the images eg for 1080x1920 DimY=1080, DimX=1920
    
    #This Currently doesn't make a median dark frame because I havent figured out a way of doing that 
    #without making this code take roughly one-million times longer to run.
    
    #FileNameList = MakeFileNamesList(datadirectory,filepattern)
    FileNameList = glob2.glob(datadirectory + filepattern)
    ImagesList = ut.MakeImagesList(FileNameList,1)

    NumberOfFrames = len(ImagesList)
    SumDark = np.zeros((DimY,DimX))
    SumDarkSquared = np.zeros((DimY,DimX))
    SumExposureTime = 0
    
    HotCounterArray = np.zeros((DimY,DimX))
    ZerosArray = np.zeros((DimY,DimX))
    #SumDark = np.zeros((1080,1920))
    
    FrameMedians = []
    FrameStds = []
    
    #making master dark frame by finding the mean
    for i in tqdm(range(len(ImagesList))):
    #for all images in list of images
        
        #For Mean and Std Dark Frames
        image_and_metadata = ImagesList[i]
        image_data = image_and_metadata[0]
        SumDark = SumDark + image_data
        SumDarkSquared = SumDarkSquared + image_data**2
        
        SumExposureTime = SumExposureTime + float(image_and_metadata[1])
        
        #For finding Hot Pixels
        FrameMedian = np.median(image_data)
        FrameStd = np.std(image_data)
        
        FrameMedians.append(FrameMedian)
        FrameStds.append(FrameStd)
        
        #hotsig is the number of stdevs above median a pixel must be to be considered hot.
        #HotThresh = FrameMedian + hotsig*FrameStd
        HotsToCount = np.where(image_data > HotThresh,1,ZerosArray)
        
        HotCounterArray = HotCounterArray + HotsToCount


    MeanDarkFrame = SumDark/len(ImagesList)
    StdDarkFrame = np.sqrt((SumDarkSquared/len(ImagesList)) - MeanDarkFrame**2)
    HotsArray = np.where((HotCounterArray/len(ImagesList)) > hotfrac,1,ZerosArray)
    
    MeanExposureTime = SumExposureTime/len(ImagesList)
    
    FrameMedians = np.array(FrameMedians)
    FrameStds = np.array(FrameStds)
    
    return MeanDarkFrame, HotsArray, FrameMedians, FrameStds

In [53]:
#Analyzing the most recent sets of Dark Frames using the same HotThres as the old 1000 ms darks
SetNamesList = ['X0_Y0','X0_YM1','X0_YM2','X0_YM3','X0_YP1','X0_YP2','X0_YP3','XM2_Y0','XM2_YM1','XM2_YP1','XP2_Y0','XP2_YM1','XP2_YP1','XP4_Y0']
DimY = 1096
DimX = 1936
hotsig = 5.0 #Number of stdevs above a frame's median a pixel must be to be considered hot.
hotfrac = 0.1 #The fraction of the frames in a given set a certain pixel must be hot in order to be recorded as hot in the HotsArray
HotThresh = 9.71
DarkFrameName = 'PrelimMeanCS_Dark'
HotsArrayName = 'PrelimHotsArray'

ThreshHotCount = []
ThreshHotFrac = []
ThreshFrameMediansList = []
ThreshFrameStdsList = []

for i in range(len(SetNamesList)):
    SetName = SetNamesList[i]
    
    #OpenDirectory
    DataDirectory = 'SteadyStateDataSets/' + SetName + '\CS_Darks/'
    filepatternOpen = '*.FTS'
    
    #SaveDirectory
    SaveDirectory = 'SteadyStateDataSets/' + SetName + ForwardSlash 
    
    #print(datadirectoryOpen + folderOpen)
    #print(datadirectorySave + folderSave)
    #print(SaveDirectory)
    MeanDarkFrame, HotsArray, FrameMedians, FrameStds = ReturnMasterDarkAndHotsArrayExtraThresh(DataDirectory,filepatternOpen,DimX,DimY,HotThresh,hotfrac)
    
    #np.savetxt(SaveDirectory + DarkFrameName +'.csv', MeanDarkFrame, delimiter=',')
    #np.savetxt(SaveDirectory + HotsArrayName +'.csv', HotsArray, delimiter=',')
    
    #print(SetName + ' Preliminary Hot Pixel Count' + ": " + str(np.sum(HotsArray)))
    
    ThreshHotCount.append(np.sum(HotsArray))
    ThreshHotFrac.append(np.sum(HotsArray)/(DimX*DimY))
    ThreshFrameMediansList.append(FrameMedians)
    ThreshFrameStdsList.append(FrameStds)
    
#Analyzing old sets of Dark Frames
ComboSetNamesList = SetNamesList

#for entry in SetNamesList:
    #ComboSetNamesList.append[entry]

DataDirectory = 'OldCS300msDark/'
filepatternOpen = '*.FTS'

MeanDarkFrame, HotsArray, FrameMedians, FrameStds = ReturnMasterDarkAndHotsArrayExtraThresh(DataDirectory,filepatternOpen,DimX,DimY,HotThresh,hotfrac)

ThreshHotCount.append(np.sum(HotsArray))
ThreshHotFrac.append(np.sum(HotsArray)/(DimX*DimY))
ThreshFrameMediansList.append(FrameMedians)
ThreshFrameStdsList.append(FrameStds)

ComboSetNamesList.append('OldCS300msDark')

DataDirectory = 'OldCS1000msDark/'
filepatternOpen = '*.FTS'

MeanDarkFrame, HotsArray, FrameMedians, FrameStds = ReturnMasterDarkAndHotsArrayExtraThresh(DataDirectory,filepatternOpen,DimX,DimY,HotThresh,hotfrac)

ThreshHotCount.append(np.sum(HotsArray))
ThreshHotFrac.append(np.sum(HotsArray)/(DimX*DimY))
ThreshFrameMediansList.append(FrameMedians)
ThreshFrameStdsList.append(FrameStds)

ComboSetNamesList.append('OldCS1000msDark')

for i in range(len(ComboSetNamesList)):
    SetName = ComboSetNamesList[i]
    ThreshMeanFrameMedian = np.mean(ThreshFrameMediansList[i])
    ThreshMeanFrameStd = np.mean(ThreshFrameStdsList[i])
    hotsig = 5.0
    Usual_HotThresh = ThreshMeanFrameMedian + ThreshMeanFrameStd*hotsig
    print("_________________________")
    print("")
    print(" Set: "+ SetName)
    print("-------------------------")
    print(" Hot Pixel Count (HotThresh = 9.71): " + str(ThreshHotCount[i]))
    print(" Hot Pixel Fraction (HotThresh = 9.71): " + str(round(ThreshHotFrac[i]*100,4))+"%")
    print(" Mean Frame Median: " + str(round(ThreshMeanFrameMedian,2)))
    print(" Mean Frame Std: " + str(round(ThreshMeanFrameStd,2)))
    print(" Usual HotThresh: " + str(Usual_HotThresh))
    
    
print("_________________________")

  0%|          | 0/246 [00:00<?, ?it/s]

C:\Users\Colin Packard\AppData\Local\Temp\ipykernel_12804\4245751307.py:104: RuntimeWarning: invalid value encountered in sqrt
  StdDarkFrame = np.sqrt((SumDarkSquared/len(ImagesList)) - MeanDarkFrame**2)


  0%|          | 0/36 [00:00<?, ?it/s]

  0%|          | 0/48 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

...

  0%|          | 0/49 [00:00<?, ?it/s]

  0%|          | 0/48 [00:00<?, ?it/s]

  0%|          | 0/99 [00:00<?, ?it/s]

  0%|          | 0/99 [00:00<?, ?it/s]

_________________________

 Set: X0_Y0
-------------------------
 Hot Pixel Count (HotThresh = 9.71): 1807.0
 Hot Pixel Fraction (HotThresh = 9.71): 0.0852%
 Mean Frame Median: 4.0
 Mean Frame Std: 3.1
 Usual HotThresh: 15.513598659765568
_________________________

 Set: X0_YM1
-------------------------
 Hot Pixel Count (HotThresh = 9.71): 2101.0
 Hot Pixel Fraction (HotThresh = 9.71): 0.099%
 Mean Frame Median: 4.0
 Mean Frame Std: 3.07
 Usual HotThresh: 15.334565014563779
_________________________

 Set: X0_YM2
-------------------------
 Hot Pixel Count (HotThresh = 9.71): 2235.0
 Hot Pixel Fraction (HotThresh = 9.71): 0.1053%
 Mean Frame Median: 4.0
 Mean Frame Std: 3.12
 Usual HotThresh: 15.583176452740013
_________________________

 Set: X0_YM3
-------------------------
 Hot Pixel Count (HotThresh = 9.71): 1762.0
 Hot Pixel Fraction (HotThresh = 9.71): 0.083%
 Mean Frame Median: 4.0
 Mean Frame Std: 3.11
 Usual HotThresh: 15.548800430783757
_________________________

 Set: X0_YP1


In [55]:
#Analyzing the most recent sets of Dark Frames using the same HotThres as the old 1000 ms darks
SetNamesList = ['X0_Y0','X0_YM1','X0_YM2','X0_YM3','X0_YP1','X0_YP2','X0_YP3','XM2_Y0','XM2_YM1','XM2_YP1','XP2_Y0','XP2_YM1','XP2_YP1','XP4_Y0']
DimY = 1096
DimX = 1936
hotsig = 5.0 #Number of stdevs above a frame's median a pixel must be to be considered hot.
hotfrac = 0.1 #The fraction of the frames in a given set a certain pixel must be hot in order to be recorded as hot in the HotsArray
HotThresh = 10.30
DarkFrameName = 'PrelimMeanCS_Dark'
HotsArrayName = 'PrelimHotsArray'

ThreshHotCount = []
ThreshHotFrac = []
ThreshFrameMediansList = []
ThreshFrameStdsList = []

for i in range(len(SetNamesList)):
    SetName = SetNamesList[i]
    
    #OpenDirectory
    DataDirectory = 'SteadyStateDataSets/' + SetName + '\CS_Darks/'
    filepatternOpen = '*.FTS'
    
    #SaveDirectory
    SaveDirectory = 'SteadyStateDataSets/' + SetName + ForwardSlash 
    
    #print(datadirectoryOpen + folderOpen)
    #print(datadirectorySave + folderSave)
    #print(SaveDirectory)
    MeanDarkFrame, HotsArray, FrameMedians, FrameStds = ReturnMasterDarkAndHotsArrayExtraThresh(DataDirectory,filepatternOpen,DimX,DimY,HotThresh,hotfrac)
    
    #np.savetxt(SaveDirectory + DarkFrameName +'.csv', MeanDarkFrame, delimiter=',')
    #np.savetxt(SaveDirectory + HotsArrayName +'.csv', HotsArray, delimiter=',')
    
    #print(SetName + ' Preliminary Hot Pixel Count' + ": " + str(np.sum(HotsArray)))
    
    ThreshHotCount.append(np.sum(HotsArray))
    ThreshHotFrac.append(np.sum(HotsArray)/(DimX*DimY))
    ThreshFrameMediansList.append(FrameMedians)
    ThreshFrameStdsList.append(FrameStds)
    
#Analyzing old sets of Dark Frames
ComboSetNamesList = SetNamesList

#for entry in SetNamesList:
    #ComboSetNamesList.append[entry]

DataDirectory = 'OldCS300msDark/'
filepatternOpen = '*.FTS'

MeanDarkFrame, HotsArray, FrameMedians, FrameStds = ReturnMasterDarkAndHotsArrayExtraThresh(DataDirectory,filepatternOpen,DimX,DimY,HotThresh,hotfrac)

ThreshHotCount.append(np.sum(HotsArray))
ThreshHotFrac.append(np.sum(HotsArray)/(DimX*DimY))
ThreshFrameMediansList.append(FrameMedians)
ThreshFrameStdsList.append(FrameStds)

ComboSetNamesList.append('OldCS300msDark')

DataDirectory = 'OldCS1000msDark/'
filepatternOpen = '*.FTS'

MeanDarkFrame, HotsArray, FrameMedians, FrameStds = ReturnMasterDarkAndHotsArrayExtraThresh(DataDirectory,filepatternOpen,DimX,DimY,HotThresh,hotfrac)

ThreshHotCount.append(np.sum(HotsArray))
ThreshHotFrac.append(np.sum(HotsArray)/(DimX*DimY))
ThreshFrameMediansList.append(FrameMedians)
ThreshFrameStdsList.append(FrameStds)

ComboSetNamesList.append('OldCS1000msDark')

for i in range(len(ComboSetNamesList)):
    SetName = ComboSetNamesList[i]
    ThreshMeanFrameMedian = np.mean(ThreshFrameMediansList[i])
    ThreshMeanFrameStd = np.mean(ThreshFrameStdsList[i])
    hotsig = 5.0
    Usual_HotThresh = ThreshMeanFrameMedian + ThreshMeanFrameStd*hotsig
    print("_________________________")
    print("")
    print(" Set: "+ SetName)
    print("-------------------------")
    print(" Hot Pixel Count (HotThresh = 10.30): " + str(ThreshHotCount[i]))
    print(" Hot Pixel Fraction (HotThresh = 10.30): " + str(round(ThreshHotFrac[i]*100,4))+"%")
    print(" Mean Frame Median: " + str(round(ThreshMeanFrameMedian,2)))
    print(" Mean Frame Std: " + str(round(ThreshMeanFrameStd,2)))
    print(" Usual HotThresh: " + str(Usual_HotThresh))
    
    
print("_________________________")

  0%|          | 0/246 [00:00<?, ?it/s]

C:\Users\Colin Packard\AppData\Local\Temp\ipykernel_12804\4245751307.py:104: RuntimeWarning: invalid value encountered in sqrt
  StdDarkFrame = np.sqrt((SumDarkSquared/len(ImagesList)) - MeanDarkFrame**2)


  0%|          | 0/36 [00:00<?, ?it/s]

  0%|          | 0/48 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

...

  0%|          | 0/49 [00:00<?, ?it/s]

  0%|          | 0/48 [00:00<?, ?it/s]

  0%|          | 0/99 [00:00<?, ?it/s]

  0%|          | 0/99 [00:00<?, ?it/s]

_________________________

 Set: X0_Y0
-------------------------
 Hot Pixel Count (HotThresh = 10.30): 782.0
 Hot Pixel Fraction (HotThresh = 10.30): 0.0369%
 Mean Frame Median: 4.0
 Mean Frame Std: 3.1
 Usual HotThresh: 19.51359865976557
_________________________

 Set: X0_YM1
-------------------------
 Hot Pixel Count (HotThresh = 10.30): 890.0
 Hot Pixel Fraction (HotThresh = 10.30): 0.0419%
 Mean Frame Median: 4.0
 Mean Frame Std: 3.07
 Usual HotThresh: 19.33456501456378
_________________________

 Set: X0_YM2
-------------------------
 Hot Pixel Count (HotThresh = 10.30): 893.0
 Hot Pixel Fraction (HotThresh = 10.30): 0.0421%
 Mean Frame Median: 4.0
 Mean Frame Std: 3.12
 Usual HotThresh: 19.583176452740013
_________________________

 Set: X0_YM3
-------------------------
 Hot Pixel Count (HotThresh = 10.30): 774.0
 Hot Pixel Fraction (HotThresh = 10.30): 0.0365%
 Mean Frame Median: 4.0
 Mean Frame Std: 3.11
 Usual HotThresh: 19.548800430783757
_________________________

 Set: X0_

***
### Making and Open Shutter Dark Frame for (0,2) 981 eV Frames

In [ ]:
DimY = 1096
DimX = 1936
hotsig = 5.0 #Number of stdevs above a frame's median a pixel must be to be considered hot.
hotfrac = 0.1 #The fraction of the frames in a given set a certain pixel must be hot in order to be recorded as hot in the HotsArray

DarkFrameName = 'PrelimMeanCS_Dark'
HotsArrayName = 'HotsArray'

SetName = 'X0_YP2_981eV'
datadirectory = 'SteadyStateDataSets/Gain_Data/' + SetName +"/CS_Darks/"
filepattern = '*.FTS'

SaveDirectory = 'SteadyStateDataSets/Gain_Data/' + SetName + ForwardSlash 

MeanDarkFrame, HotsArray = ut.ReturnMasterDarkAndHotsArray(datadirectory,filepattern,DimX,DimY,hotsig,hotfrac)


np.savetxt(SaveDirectory + DarkFrameName +'.csv', MeanDarkFrame, delimiter=',')
np.savetxt(SaveDirectory + HotsArrayName +'.csv', HotsArray, delimiter=',')

In [26]:
# Making an OS Dark Frame fro 981 eV data
SetName = 'X0_YP2_981eV'

Directory1 = 'SteadyStateDataSets/Gain_Data/' + SetName + ForwardSlash 
Directory2 = 'SteadyStateDataSets/Gain_Data/' + SetName + '/CroppedROI1/'
Directory3 = Directory1 + "BeamImages/"
Directory4 = Directory2 + "BeamImages/"

DarkFrameName = 'PrelimMeanCS_Dark'
HotsArrayName = 'HotsArray'
MeanDarkFrame = np.loadtxt(Directory1 + DarkFrameName + '.csv', delimiter=',')
HotsArray = np.loadtxt(Directory1 + HotsArrayName + '.csv', delimiter=',')

IntegrationRegionSize = 5
thres = 20

filepattern = '*.FTS'
FileNamesList = glob2.glob(Directory3 + filepattern)

ImagesList = ut.MakeImagesList(FileNamesList,1)

OS_MasterDarkFrame, N_Array, SampleStdFrame, StandardErrorOfMeanFrame, SumDarkArray, DivideByArray = ut.MakeMasterIlluminatedDarkFrameStandard(ImagesList,
                                                                MeanDarkFrame,HotsArray,DimX,
                                                                DimY,IntegrationRegionSize,thres)

np.savetxt(Directory2 + 'OS_MasterDarkFrame' +'.csv', OS_MasterDarkFrame, delimiter=',')

  0%|          | 0/298 [00:00<?, ?it/s]

C:\Users\Colin Packard\PythonWorkspace\Argonne12_01\Utility3_4.py:523: RuntimeWarning: invalid value encountered in sqrt
  SampleStdFrame = np.sqrt(MeanSquaredDark - (OS_MeanDarkFrame**2))
